# Processing the final dataset



In [1]:
import ee
import geemap

# Authenticate to Earth Engine
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

data_folder = "projects/amazon-forest-regrowth/assets"

roi = ee.FeatureCollection(f"{data_folder}/raw/biomes_br") \
    .filter(ee.Filter.eq("CD_Bioma", 1)).geometry()

In [24]:
# biomass_all = ee.Image(f"{data_folder}/raw/biomass")
# proj = biomass_all.projection().getInfo()

# mature_biomass = ee.Image(f"{data_folder}/mature_biomass_1km").reproject(
#     crs = proj["crs"], crsTransform = proj["transform"]
# )

# scale = biomass_all.projection().nominalScale().getInfo()

mature_biomass_1k = ee.Image(f"{data_folder}/mature_biomass_1km_100m")#.clip(roi)

task = ee.batch.Export.image.toDrive(
    image = mature_biomass_1k,
    description = 'mature_biomass_1k',
    region = mature_biomass_1k.geometry(),
    scale = 1000,
    maxPixels = 1e13
)
task.start()


In [23]:
age_agbd = ee.Image(f"{data_folder}/age_agbd")

distance = mature_biomass_1k.fastDistanceTransform().sqrt() \
    .multiply(ee.Image.pixelArea().sqrt()).rename("distance") \
    .updateMask(age_agbd.select("age")).float()#.clip(roi)

task = ee.batch.Export.image.toDrive(
    image = distance,
    description = 'distance',
    region = distance.geometry(),
    scale = 1000,
    maxPixels = 1e13
)
task.start()

In [21]:
years = 10

if (years == 0):
    land_use_years = ee.Image(f"{data_folder}/land_use")
    name = "all_LULC_mat_dist"
else:
    land_use_years = ee.Image(f"{data_folder}/land_use_{years}_years")
    name = f"{years}y_LULC_mat_dist"

age_agbd = ee.Image(f"{data_folder}/age_agbd")
categorical = ee.Image(f"{data_folder}/categorical")
sur_cover = ee.Image(f"{data_folder}/sur_cover").rename("sur_cover")
one_hectare_mask = ee.Image(f"{data_folder}/one_hectare_mask").rename("one_hectare_mask")
cwd = ee.Image(f"{data_folder}/raw/cwd_chave").float().rename("cwd")
yearly_mean_prec = ee.Image(f"{data_folder}/yearly_mean_prec")
yearly_SI = ee.Image(f"{data_folder}/yearly_SI")
mat = ee.Image(f"{data_folder}/nearest_mat_biomass_500m_amazon").rename("distance", "mature_biomass")

combined_mask = one_hectare_mask.And(land_use_years.select("last_LU")) \
    .And(mat.select("distance"))

unified_data = (
    age_agbd.addBands(
        [mat, distance, categorical, land_use_years, cwd, sur_cover, yearly_mean_prec, yearly_SI
    ]).updateMask(combined_mask)
)


unified_data_fc = unified_data.stratifiedSample(
    numPoints = 5000, classBand = 'ecoreg', region = unified_data.geometry()
)


# Export an ee.FeatureCollection to Google Drive, adjusting the description based on the years
task = ee.batch.Export.table.toDrive(
    collection=unified_data_fc, description=name
)
task.start()